In [ ]:
from transformers import LongformerModel, LongformerTokenizer, LongformerConfig
import torch
import pandas as pd
import numpy as np

In [ ]:
csv_file_path = "/kaggle/input/all-v1-transpose/all_v1_transpose.csv"  
df = pd.read_csv(csv_file_path)
df.head()

In [ ]:
input_texts = df["original_text"].tolist()

In [ ]:
def calculte_inference(model, tokenizer, input_texts, device):
    # INIT LOGGERS
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions = 300
    timings = np.zeros((repetitions, 1))

    # GPU-WARM-UP
    for _ in range(10):
        for text in df["original_text"]:
            input_ids = tokenizer(text, return_tensors="pt").to(device)
            _ = model(**input_ids)

    # MEASURE PERFORMANCE
    with torch.no_grad():
        for rep in range(repetitions):
            starter.record()
            for text in df["original_text"]:
                input_ids = tokenizer(text, return_tensors="pt").to(device)
                _ = model(**input_ids)
            ender.record()
            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time

    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    return mean_syn, std_syn

### Longformer

In [ ]:
#Longformer configuration
config = LongformerConfig.from_json_file("/kaggle/input/config-and-models/Config and Model/Longformer_config.json")
model = LongformerModel(config)

In [ ]:
# learned parameters (weights)
model.load_state_dict(torch.load("/kaggle/input/config-and-models/Config and Model/longformer_pytorch_model"))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/led-base-16384")

In [ ]:
mean_time, std_time = measure_inference_time(model, tokenizer, input_texts, device)